In [70]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from scipy import stats
import ast 

In [71]:
categories = ['banana','beach','car','church','lighthouse','mountain','mug','plane']
path='../Analysis/human_rating_bycat'

In [72]:
for i, cate in enumerate(categories):
    col = [cate+'_'+str(i+1) for i in range(16)]
    new = pd.read_csv(path+'/'+cate+'.csv')
    new.columns = col
    if i == 0:
        old = new
    else:
        old = pd.concat([old,new],axis=1)
df = old

(Test) Use quantiles to remove outliers 

In [73]:
quantile_low = df.quantile(0.1)
quantile_high = df.quantile(0.9)
iqr = quantile_high - quantile_low

In [74]:
outliers = (df < quantile_low) | (df > quantile_high)
df_new = df[outliers.sum(axis=1) <= 12]

Compute 7 scorse and sort images by order in survey

In [75]:
# alexnet_df=pd.read_csv('alexnet_scores_wordnet_id_closest_label.csv',index_col=0)
# alexnet_df['typicality_score'] = np.round(alexnet_df['closest_prob'] * 100,2)
# alexnet_df['7_score']= np.round(alexnet_df['typicality_score']*7/100,2)
# alexnet_df['type'] = alexnet_df['type'].apply(lambda x: x.lower())

# alexnet_df=pd.read_csv('alexnet_scores_wordnet_id_matched_label.csv',index_col=0)
# alexnet_df['typicality_score'] = np.round(alexnet_df['matched_prob'] * 100,2)
# alexnet_df['7_score']= np.round(alexnet_df['typicality_score']*7/100,2)
# alexnet_df['type'] = alexnet_df['type'].apply(lambda x: x.lower())

alexnet_df=pd.read_csv('alexnet_scores_final.csv',index_col=0)
alexnet_df['typicality_score'] = np.round(alexnet_df['prob'] * 100,2)
alexnet_df['7_score']= np.round(alexnet_df['typicality_score']*7/100,2)
alexnet_df['type'] = alexnet_df['type'].apply(lambda x: x.lower())

In [76]:
alexnet_df['number'] = alexnet_df['dir'].apply(lambda x: int(x.split('\\')[-1].split('.')[0]))
alexnet_df = alexnet_df.sort_values(['type','number']).reset_index(drop=True)

In [77]:
# alexnet_df['readable_labels'].apply(lambda x: ast.literal_eval(x)[list(ast.literal_eval(x).keys())[0]])

# Correlation with human ratings

Changes from original script: 
1. Use spearman rank correlation instead of pearson
2. Compute average human ratings first, and then compute correlation with neural net typicality scores

without dropping outliers

In [78]:
final_corr = dict()
for t in np.unique(alexnet_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    human_ratings = df[col_names] 
    net_ratings = alexnet_df[alexnet_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
#     net_scores = net_ratings['readable_labels'].apply(lambda x: ast.literal_eval(x)[list(ast.literal_eval(x).keys())[0]]).values
#     net_scores = net_ratings['7_score'].values # both get worse scores

    net_scores = net_ratings['typicality_score'].values
    
    coef, p=stats.spearmanr(human_scores, net_scores)
#     coef, p=stats.pearsonr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.5827816147282076
beach:  0.5588235294117647
car:  0.3970588235294118
church:  0.30757918555099845
lighthouse:  0.6911764705882353
mountain:  0.49006635783962915
mug:  0.07658329365862818
plane:  0.6897570037935065


In [79]:
print(np.mean(list(final_corr.values())))

0.4742282848875477


after dropping outliers

In [80]:
final_corr = dict()
for t in np.unique(alexnet_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    
    human_ratings = df_new[col_names] 
    net_ratings = alexnet_df[alexnet_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
    coef, p=stats.spearmanr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.5062547360265238
beach:  0.5670109242033047
car:  0.46097251759905034
church:  0.31493753927231416
lighthouse:  0.6612672856293087
mountain:  0.538590487883415
mug:  -0.0029498653417206555
plane:  0.6691292380905453


In [81]:
print(np.mean(list(final_corr.values())))

0.4644016079203427
